The notebook is to run in jetbot to benchmark inference speed. To be consise, the proces should include pre & post functions

# Import Library



In [0]:
# Importing the Keras libraries and packages
import tensorflow as tf
# from tensorflow.python.keras.models import Sequential
import tensorflow.keras
from tensorflow.keras.models import load_model, model_from_yaml
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import IPython.display as display
from pathlib import Path
import numpy as np
import datetime as dt
from PIL import Image
import os, sys, cv2, random, math, pickle, time

# Define Processing Functions

In [0]:
def convertToOneHot(left,right):
    if left == right:
        return [0,1,0]
    elif left > right: # Turn Right
        return [0,0,1]
    elif right > left: # Turn Left
        return [1,0,0]

## This function have to deploy to jetbot
def preprocess(image, h=224, w=224, crop_top=0.0, color=False): 

    """
    The function is to preprocess image before feeding into model. 
    Current feature include resize image based on h and w, vertical cropping, and change color.
    
    :param image: input imag
    :param h: target height of output image in pixel. Default to 224
    :param w: target width of output image in pixel. Default to 224
    :param crop_top: Determine whether to crop the top part of the image. use ratio value. e.g. 0.5 mean crop top half of image. Default to 0
    :param color: cv2 color conversion. Put False if want to keep original color. Default to cv2.COLOR_RGB2YUV. Possible values [cv2.COLOR_BGR2GRAY,cv2.COLOR_RGB2YUV,cv2.COLOR_RGB2GRAY,cv2.COLOR_RGB2GRAY, cv2.COLOR_BGR2RGB, cv2.COLOR_RGB2BGR]
    :return normalized and processed image
    
    """

    height, _, _ = image.shape
    image = image[int(height*crop_top):,:,:]  # remove top half of the image, as it is not relevant for lane following
    if color:
      image = cv2.cvtColor(image, color) 
    image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (w,h)) # input image size (200,66) Nvidia model
    image = image / 255 # normalizing

    return image
  
def process_prediction(prediction):
    predicted_index = np.argmax(prediction)
    result = np.zeros(len(prediction))
    result[predicted_index] = 1
    return result


# Load Sample Data

In [0]:
## Start To Change ##

BASE_DIR = os.getcwd()
EXP_MODEL_DIR = os.path.join(BASE_DIR, 'models')
RAW_DATA_DIR  = os.path.join(BASE_DIR,'data') ## Change Here

## End To Change ##

In [0]:
data_x = [] ## Input (224)
data_y = [] ## Label (motor left, motor right)
SAMPLE_SIZE = 1000
## Load data from raw data folder
## Data is split to 2 differant arrays

for obj in os.listdir(RAW_DATA_DIR):
  if obj.endswith('.npy'):
    data = np.load(os.path.join(RAW_DATA_DIR,obj),allow_pickle=True)
    data = data[((data[:,0] > 0.0) & (data[:,1] > 0.0))]
    print(f'Loaded : {obj} for {data.shape}' )
    data_x.extend(data[:SAMPLE_SIZE,2])
    data_y.extend(data[:SAMPLE_SIZE,0:2])

data_x = np.array(data_x)
data_y = np.array(data_y)
print(f'All Input: {data_x.shape} \t All Label: {data_y.shape}')

Loaded : training_data_all_2020_Feb_14_00_05_59.npy for (6330, 3)
All Input: (1000, 224, 224, 3) 	 All Label: (1000, 2)


Define some parameters

In [0]:
IMG_H = 66 ## Change Here
IMG_W = 200 ## Change Here
T_CROP = 0.30 ## Change Here
COLOR_CONV = cv2.COLOR_BGR2YUV ## Change Here

# HDF5

In [0]:
HDF5_PATH = os.path.join(EXP_MODEL_DIR, 'Pilotnet_CAT_YUV_50E_TF114_model.h5') ## Change here
model = load_model(HDF5_PATH)
print(f"Loaded best checkpoint from disk: {HDF5_PATH}")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Loaded best checkpoint from disk: /home/jetbot/ITI110_Project/models/Pilotnet_CAT_YUV_50E_TF114_model.h5


In [0]:
start_time = time.time()
x = 1 # displays the frame rate every 1 second
counter = 0

for sample in data_x:
  
  sample = preprocess(sample,IMG_H,IMG_W,T_CROP,COLOR_CONV)
  h, w, c = sample.shape
  sample = sample.reshape(1,h,w,c)
  output_data = process_prediction(model.predict(sample, verbose=0)[0]).astype('int32').tolist()

  counter+=1
  if (time.time() - start_time) > x :
    print("FPS: ", counter / (time.time() - start_time))
    counter = 0
    
    start_time = time.time()

FPS:  32.665664597627746
FPS:  32.40672334228499
FPS:  33.81183152877221
FPS:  29.82195577749627
FPS:  32.087411121077025
FPS:  33.59468238180898
FPS:  32.921931854670866
FPS:  32.592744553258015
FPS:  33.11960663273103
FPS:  33.35612395670259
FPS:  32.16631327079729
FPS:  32.95057587006682
FPS:  33.363131727747415
FPS:  31.82929986212287
FPS:  31.759293512944463
FPS:  33.9859091053166
FPS:  32.59137849311414
FPS:  33.74914471255337
FPS:  33.08433857926479
FPS:  32.261342640449925
FPS:  32.04344936267091
FPS:  33.85640725406585
FPS:  33.11080179143304
FPS:  28.996128599163168
FPS:  33.70179209470492
FPS:  32.456680195360974
FPS:  32.28182381113976
FPS:  34.2017957728349
FPS:  32.20915139754389
FPS:  32.652063530107824


# TFLITE

In [0]:
TFLITE_PATH = os.path.join(EXP_MODEL_DIR, 'Pilotnet_CAT_YUV_50E_TF114_model.tflite') ## Change here 

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=TFLITE_PATH)
print(f'Loaded Model: {TFLITE_PATH}')
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
print(input_details)
output_details = interpreter.get_output_details()

Loaded Model: /home/jetbot/ITI110_Project/models/Pilotnet_CAT_YUV_50E_TF114_model.tflite
[{'name': 'conv2d_input', 'index': 20, 'shape': array([  1,  66, 200,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]


In [0]:
start_time = time.time()
x = 1 # displays the frame rate every 1 second
counter = 0

for sample in data_x:
  
  sample = preprocess(sample,IMG_H,IMG_W,T_CROP,COLOR_CONV)
  h, w, c = sample.shape
  sample = sample.reshape(1,h,w,c)
  sample = np.array(sample, dtype=np.float32) ## Change dtype based on input details print out from cell above
  interpreter.set_tensor(input_details[0]['index'], sample)
  interpreter.invoke()
  output_data = process_prediction(interpreter.get_tensor(output_details[0]['index'])[0]).astype('int32').tolist()

  counter+=1
  if (time.time() - start_time) > x :
    print("FPS: ", counter / (time.time() - start_time))
    counter = 0
    start_time = time.time()

FPS:  35.74286935628897
FPS:  41.651597819826414
FPS:  40.23764118386315
FPS:  44.76568559134664
FPS:  33.845085631856826
FPS:  39.59459545568363
FPS:  43.162649716196725
FPS:  38.93639179311599
FPS:  39.56053636765996
FPS:  43.712988853446234
FPS:  38.87827551031079
FPS:  39.71584491963545
FPS:  43.98854744581918
FPS:  38.93877381960385
FPS:  39.977553572973164
FPS:  40.835055468144105
FPS:  39.549783713063256
FPS:  39.746056835395116
FPS:  44.56259784916584
FPS:  40.267914403691506
FPS:  40.70532027605483
FPS:  43.61998040580546
FPS:  39.63582851328458
FPS:  41.31281303711875
